In [1]:
from pathlib import Path
import os
from dotenv import load_dotenv

%load_ext autoreload
%autoreload 2

In [2]:
# change dir to ml repo at ml/ root (if you started in notebook/)
os.chdir('../')
print(Path.cwd())
load_dotenv()

/home/opadmin/llama2d_opentable/lm_act_eval


True

In [3]:
# read dataset into datasets
from datasets import Dataset
import pandas as pd
dataset_name = "five-star-trajectories"
dataset_path = Path(f"lm_act_eval/.cache/{dataset_name }")
traj_df = pd.read_csv(dataset_path/"csv/data.csv")
traj_dataset = Dataset.from_pandas(traj_df)
print(traj_df.columns)
traj_df[traj_df.session_id==traj_df.session_id.unique()[0]]
traj_df.head(2)

/home/opadmin/anaconda3/envs/eval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Index(['session_id', 'idx_in_session', '_id', 'event_id', 'parent_id',
       'event_type', 'event_name', 'config', 'children_ids', 'inputs',
       'outputs', 'error', 'start_time', 'end_time', 'metadata', 'feedback',
       'metrics', 'user_properties', 'source', 'duration', 'project_id',
       'tenant', 'QUERY', 'URL', 'DOM', 'RULES', 'USER_CONTEXT',
       'PREV_ACTIONS', 'chat_history', 'CURRENT_TIME', 'planPrompt',
       'previousActionsRepetitionPrompt', 'chat_completion_messages',
       'ground_truth', 'screenshot', 'prompt', 'internal_metadata'],
      dtype='object')


,session_id,idx_in_session,_id,event_id,parent_id,event_type,event_name,config,children_ids,inputs,...,PREV_ACTIONS,chat_history,CURRENT_TIME,planPrompt,previousActionsRepetitionPrompt,chat_completion_messages,ground_truth,screenshot,prompt,internal_metadata
0,ce3bf1c8-371a-4adb-98e6-2cce19c1aff5,0,65f0c5d705cef8bdb70153a9,cc3d0d22-4759-43d8-a688-d174ca94f9dd,ce3bf1c8-371a-4adb-98e6-2cce19c1aff5,model,Model Completion,"{'model': 'gpt-4-1106-preview', 'provider': 'u...",[],{'QUERY': 'Find a restaurant with vegetarian o...,...,NaN,[],"Tuesday, March 12, 2024 at 4:14:11 PM CDT","IMPORTANT: If the objective is complex, compri...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...",PLAN: To find a restaurant with vegetarian opt...,https://multion-client-screenshots.s3.us-east-...,NaN,NaN
1,ce3bf1c8-371a-4adb-98e6-2cce19c1aff5,1,65f0c60705cef8bdb70153ab,44b78fc2-24e2-40e5-b39d-e887a7129587,ce3bf1c8-371a-4adb-98e6-2cce19c1aff5,model,Model Completion,"{'model': 'gpt-4-1106-preview', 'provider': 'u...",[],{'QUERY': 'Find a restaurant with vegetarian o...,...,- I am searching for restaurants with vegetari...,"[{'role': 'user', 'content': 'Find a restauran...","Tuesday, March 12, 2024 at 4:15:02 PM CDT","IMPORTANT: If the objective is complex, compri...",**PREVIOUS ACTIONS Repetition**:\n- Review the...,"[{'role': 'system', 'content': '**Goal**: Let\...","The search for ""vegetarian"" in St. Louis has y...",https://multion-client-screenshots.s3.us-east-...,NaN,NaN


In [4]:
from lm_act_eval.evaluation_harness.helper_functions.multion import (
  action_prefix,
  clean_extracted_text,
  extract_thought,
  extract_action,
  extract_explanation,
  ParseChatCompletion
)
from lm_act_eval.evaluation_harness.utils.url import is_screenshot_url_accessible
from typing import Callable
from tqdm import tqdm
tqdm.pandas()


Google Cloud not set up, skipping import of providers.gemini_utils.generate_from_gemini_completion
Google Cloud not set up, skipping import of vertexai.preview.generative_models.Image and llms.generate_from_gemini_completion


/home/opadmin/anaconda3/envs/eval/lib/python3.11/site-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Mapping[str, gymnasium.spaces.space.Space[typing.Any]] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


### Load other data

In [5]:
traj_df = pd.read_csv(dataset_path/'csv/data+gptv.csv')

### completions

In [6]:
from lm_act_eval.evaluation_harness.constants import BNB_QUANTIZATION_CONFIG_8BIT, BitsAndBytesConfig

from lm_act_eval.common.hf import load_model_and_tokenizer
from lm_act_eval.evaluation_harness.evaluators.sft.utils import generate_text_and_merge

In [ ]:
from concurrent.futures import ProcessPoolExecutor
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer
import torch

# Check for GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "MULTION-AI/feasible-morning-17-v4"
# Load the tokenizer and model
bnb = True
tokenizer = AutoTokenizer.from_pretrained(model_name)
if not bnb:
  model = AutoModelForCausalLM.from_pretrained(model_name)
else:
  quantization_config = BitsAndBytesConfig(load_in_8bit_fp32_cpu_offload=True)
  model = AutoModelForCausalLM.from_pretrained(
    model_name, low_cpu_mem_usage=True, 
    device_map="auto", quantization_config=quantization_config, 
    max_memory={**{k:'8000Mib' for k in range(0,8)}, "cpu": '100Gib'}
  )

Loading checkpoint shards: 100%|██████████| 19/19 [00:46<00:00,  2.46s/it]


In [ ]:
traj_df_generated = generate_text_and_merge(
    traj_df,
    "chat_completion_messages_content",
    model,
    tokenizer,
    model.device,
    max_new_tokens=256,
    batch_size=2
)

In [ ]:
traj_df_new_gen = traj_df.copy()
traj_df_new_gen.to_csv(dataset_path/f'csv/data+gen_{model_name}.csv', index=False)

In [25]:
# def generate_respsonse(
#     row, pipeline, inp_column="completion_msg_content", log=False, max_new_tokens=256):
#     response = pipeline(
#         row[inp_column], max_new_tokens=max_new_tokens, num_return_sequences=1)
#     generated_text = response[0]["generated_text"]
#     if log:
#         wandb.log({
#             "original_text": row[inp_column],
#             "generated_text": generated_text})
#     return generated_text

# # responses = []
# # for index, row in tqdm(traj_df.iterrows(), desc="generating completions"):
# #     responses.append(generate_respsonse(
# #         row, inp_column="completion_msg_content", log=False
# #     ))
# # traj_df['generated_responses'] = responses


Logging - generation

In [ ]:
import wandb
from dotenv import load_dotenv
load_dotenv()
# Initialize Wandb
wandb.login()
run = wandb.init(
  project="trajectory_eval", entity="multion-agi",
  name=f"opentable-{model_name}", reinit=True)

In [ ]:
opentable_artifact = wandb.Artifact(f"opentable_trajectories_eval-{model_name}", type="dataset")
# opentable_table = wandb.Table(dataframe=traj_df_new)
eval_generation = wandb.Table(dataframe=traj_df)
#
# opentable_artifact.add(opentable_table, "opentable")
opentable_artifact.add(eval_generation, "eval-generations")


In [ ]:
# Log the table to visualize with a run...
run.log({"eval-generations": eval_generation})
# and Log as an Artifact to increase the available row limit!
run.log_artifact(opentable_artifact)

### Process data

### Eval Metrics

In [ ]:
from autoevals.llm import *
from autoevals.string import Levenshtein
import warnings
levenshtein_evaluator = Levenshtein()

0.8

In [ ]:
config_metrics = {
    "Actions": {"edit": "edit"}
}
metric_registry = {
    "edit": Levenshtein()
}
def evaluate_trajectory(dataset):
    """
    eligible dataset
    """
    metric_results = []
    for index, row in dataset[eligible].iterrows():
        result_row = {}
        for metric_config in config_metrics:
            metric_name = metric_config["name"]
            if metric_registry.get(metric_name):
                metric_func = metric_registry[metric_name]
                result_row[metric_name] = metric_func(row)
        metric_results.append(result_row)

### Logging

In [ ]:
# log data
import wandb
from dotenv import load_dotenv
load_dotenv()
wandb.login(relogin=True)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/opadmin/.netrc


True

In [ ]:
opentable_artifact = wandb.Artifact("opentable_trajectories_gptv.v0", type="dataset")
# opentable_table = wandb.Table(dataframe=traj_df_new)

opentable_table_eligible = wandb.Table(dataframe=eligible_traj_df)
#
# opentable_artifact.add(opentable_table, "opentable")
opentable_artifact.add(opentable_table_eligible, "opentable-eligibleonly")
# opentable_artifact.add_file(str(dataset_path/'csv/data+gptv.csv'))
opentable_artifact.add_file(str(dataset_path/'csv/data+gptv-eligible.csv'))

# Log the table to visualize with a run...
run.log({"opentable_gptv_generation": opentable_table_eligible})
